### Part1:
I will use the following Ranking Metrics:
1. Precision@k - it will provide insights on how precisely our ranking model works among some part of the documents.
2. Recall@k - it will provide insights on how fully our ranking model display results to the user.
3. Hits@k - it will provide insights on how relevant the ranking for real user.

### Part 2:
- Assumption: score $\geq$ 0.65 is represent a relevant document.
- Note: all scores are in [0, 1]
___
Functions that will generate search results from the following distribution:
1. Normal(Gaussian) - with mean=0.6 and standard deviation=0.2 represent easy query where more documents will be relevant
2. Uniform - no guarantees on how many relevant documents will be
3. Beta - with a=2 and b=3 most of the documents will be around score 0.4 what represents more difficult query and less relevant results

In [20]:
import numpy as np
from scipy.stats import norm, uniform, beta

DISTRIBUTION_DICT = {'norm':{'loc':0.6, 'scale':0.2},'uniform':{}, 'beta':{'a':2, 'b':3}}

class Distribution:
    def __init__(self, name:str):
        if name in DISTRIBUTION_DICT.keys():
            self.name = name
        else:
            raise NameError("No such distribution")
    
    def sample_elements(self, n:int):
        if n < 0:
            raise ValueError("Negative sample size")
        
        elif self.name == 'norm':
            return np.clip(norm.rvs(loc=DISTRIBUTION_DICT[self.name]['loc'], scale=DISTRIBUTION_DICT[self.name]['scale'], size=n), 0, 1)
        
        elif self.name == 'uniform':
            return uniform.rvs(size=n)
        
        elif self.name == 'beta':
            return beta.rvs(a=DISTRIBUTION_DICT['beta']['a'], b=DISTRIBUTION_DICT['beta']['b'], size=n)
        

### Part 3:
Implementing class Ranking with all ranking functions
Assumption: half of the results will be displayed on the first page and third part of the results will be displayed on the screen without scrolling.

In [28]:
class Ranking:
    def __init__(self, results:np.ndarray):
        self.threshold = 0.65
        self.results = np.where(results >= self.threshold, 1, 0)
        self.ranking_args = {'Prec@k': 0.5, 'Recall@k': 0.5, 'Hit@k':0.3}
        
    def precision(self):
        k = int(len(self.results)*self.ranking_args['Prec@k'])
        return sum(self.results[:k])/k   
    
    def recall(self):
        k = int(len(self.results)*self.ranking_args['Recall@k'])
        return sum(self.results[:k])/sum(self.results)
        
    def hits(self):
        k = int(len(self.results)*self.ranking_args['Hit@k'])
        return 1 if sum(self.results[:k]) > 0 else 0
    


### Part 4:
Evaluating rank metrics

In [33]:
norm_d = Distribution('norm').sample_elements(20)
uniform_d = Distribution('uniform').sample_elements(20)
beta_d = Distribution('beta').sample_elements(20)


norm_ranking = Ranking(norm_d)
print(f"For normal distribution:\nInterpreted values:{norm_ranking.results}\nPrecission@k:{norm_ranking.precision()}\nRecall@k:{norm_ranking.recall()}\nHits@k:{norm_ranking.hits()}")
print('_'*50)
uniform_ranking = Ranking(uniform_d)
print(f"For uniform distribution:\nInterpreted values:{uniform_ranking.results}\nPrecission@k:{uniform_ranking.precision()}\nRecall@k:{uniform_ranking.recall()}\nHits@k:{uniform_ranking.hits()}")
print('_'*50)
beta_ranking = Ranking(beta_d)
print(f"For beta distribution:\nInterpreted values:{beta_ranking.results}\nPrecission@k:{beta_ranking.precision()}\nRecall@k:{beta_ranking.recall()}\nHits@k:{beta_ranking.hits()}")

For normal distribution:
Interpreted values:[1 1 1 0 1 0 0 1 1 0 0 1 1 1 0 0 0 1 1 0]
Precission@k:0.6
Recall@k:0.5454545454545454
Hits@k:1
__________________________________________________
For uniform distribution:
Interpreted values:[0 0 0 1 1 1 0 0 1 1 0 1 0 0 0 0 0 1 1 0]
Precission@k:0.5
Recall@k:0.625
Hits@k:1
__________________________________________________
For beta distribution:
Interpreted values:[0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0]
Precission@k:0.1
Recall@k:0.3333333333333333
Hits@k:0
